# CLASSIFICATORE SVM LINEARE CON INPUT DI WORD EMBEDDINGS

Classificatore basato su svm lineare che prende in input i word embeddings di parole italiane, più specificamente dei word embedding sviluppati per EvalIta 2018 dall'Italian NLP Lab. 

***

In [19]:
import numpy as np 
import re 
import os

### Caricamento Word Embeddings

Il file txt dei word embedding è stato ottenuto attraverso l'elaborazione di un file sqlite. Per ogni riga (la quale rappresenta una parola) all'interno del file, si crea un record in un dizionario con chiave parola e come valore il vattore che rappresenta il word embedding.

In [20]:
#funzione per il caricamento degli embedding itwac
def load_embeddings():
    embeddings_dict = dict()
    for line in open('../data/embeddings/itwac32.txt', encoding='utf-8'):
        splitted = line.strip().split('\t')
        word = splitted[0]
        embedding = splitted[1:]
        embedding = [float(val) for val in embedding]
        embeddings_dict[word] = np.asarray(embedding)
    return embeddings_dict

embeddings = load_embeddings()


In [21]:
embeddings['amico']

array([-0.02069134,  0.09119736,  0.25785723, -0.23561105, -0.28197852,
       -0.13193955, -0.13197723, -0.05229089, -0.28881341,  0.06564969,
       -0.30802506,  0.11779311, -0.03571652, -0.08748714, -0.24729131,
        0.2577146 ,  0.11925782, -0.27795964,  0.20498367,  0.08414506,
        0.08175091, -0.05181665, -0.34403449, -0.05261306,  0.08858071,
       -0.09748928,  0.12879393,  0.04387102, -0.04690454,  0.08181785,
        0.321078  ,  0.01658573])

### Normalizzazione del testo 

Le parole all'interno dei post devono essere normalizzate in questo modo:

Numeri:
- interi tra 0 e 2100 possono essere mantenuti così come sono
- i numeri interi maggiori di 2100 diventano una stringa specifica con un numero che rappresenta il numero di cifre
- se non si tratta di numeri interi, si convertono le cifre all'interno della stringa in questa sequenze @Dg

Parole:
- le parole che iniziano con una lettera maiuscola devono avere prima parola maiuscola e le altre minuscole
- le parole che iniziano con una lettera minuscola devono avere tutti i caratteri minuscolo

In [22]:
#funzione che trasforma le cifre all'interno di un token
def digit_norm(word):
    try:
        val = int(word)
    except:
        normalized = re.sub('\d', '@Dg', word)
        return normalized
    if val >= 0 and val<1200:
        return str(val)
    else:
        return f'DIGLEN_{str(len(str(val)))}'

#funzione che normalizza i token
def normalize(word):
    if "http" in word or ("." in word and "/" in word):
        return str("___URL___")
    if len(word)>26:
        return "__LONG-LONG__"
    digits = digit_norm(word)
    if digits != word:
        word = digits
    if word[0].isupper():
        word = word.capitalize()
    else:
        word = word.lower()
    return word



In [23]:
#funzione che estrae dai file conllu i token e li normalizza
def get_tokens(doc_path):
    doc_tokens = []
    skip_lines = 0
    first = False
    for line in open(doc_path, "r", encoding="utf-8"):
        splitted = line.strip().split("\t")
        if line[0].isdigit():
            if skip_lines == 0 and first == False:
                if "-" not in splitted[0]:
                    word = normalize(splitted[1])
                    pos=splitted[3]
                    token = {'word': word, 'pos': pos}
                    doc_tokens.append(token)
                else:
                    word=normalize(splitted[1])
                    skip_lines = len(splitted[0].split('-'))
                    first = True
            elif skip_lines != 0 and first == True:
                pos = splitted[3]
                token = {'word': word, 'pos': pos}
                doc_tokens.append(token)
                skip_lines-=1
                first = False
            else: 
                skip_lines-=1
    return doc_tokens

#funzione che crea il traingset o test set in base all'argomento specificato
def create_set(type):
    annotated_posts = []
    for doc in os.listdir("../data/UD_annotation"):
        if type in doc: 
            doc_path = "../data/UD_annotation/" + doc    
            doc_tokens = get_tokens(doc_path)
            annotated_posts.append(doc_tokens)
    return annotated_posts


print(get_tokens("../data/UD_annotation/training#125642756147265536#0#0#TW-SENTIPOLC.conllu"))

training_set = create_set("training")
test_set = create_set("test")

[{'word': 'Le', 'pos': 'DET'}, {'word': '5', 'pos': 'NUM'}, {'word': 'sgradevoli', 'pos': 'ADJ'}, {'word': 'realtà', 'pos': 'NOUN'}, {'word': 'di', 'pos': 'ADP'}, {'word': 'cui', 'pos': 'PRON'}, {'word': 'Berlusconi', 'pos': 'PROPN'}, {'word': 'dovrebbe', 'pos': 'AUX'}, {'word': 'rendersi', 'pos': 'VERB'}, {'word': 'personalmente', 'pos': 'ADV'}, {'word': 'conto', 'pos': 'NOUN'}, {'word': '<url>', 'pos': 'PROPN'}, {'word': 'Mario', 'pos': 'PROPN'}, {'word': 'Monti', 'pos': 'PROPN'}, {'word': 'non', 'pos': 'ADV'}, {'word': 'usa', 'pos': 'VERB'}, {'word': 'mezzi', 'pos': 'ADJ'}, {'word': 'termini', 'pos': 'NOUN'}]


In [24]:
training_set[2]

[{'word': '#la@dg', 'pos': 'NUM'},
 {'word': 'ma', 'pos': 'CCONJ'},
 {'word': 'perche', 'pos': 'ADV'},
 {'word': "'", 'pos': 'PUNCT'},
 {'word': 'Mario', 'pos': 'PROPN'},
 {'word': 'Monti', 'pos': 'PROPN'},
 {'word': 'non', 'pos': 'ADV'},
 {'word': 'fa', 'pos': 'VERB'},
 {'word': 'il', 'pos': 'DET'},
 {'word': 'premier', 'pos': 'NOUN'},
 {'word': '?', 'pos': 'PUNCT'},
 {'word': 'Che', 'pos': 'DET'},
 {'word': 'persona', 'pos': 'NOUN'},
 {'word': 'competente', 'pos': 'ADJ'},
 {'word': 'e', 'pos': 'CCONJ'},
 {'word': 'per', 'pos': 'ADP'},
 {'word': 'bene', 'pos': 'ADV'},
 {'word': '!', 'pos': 'PUNCT'}]

### Pre-processing

Per fare in modo di avere un vettore di features per ogni sample del nostro dataset, per ogni post vanno estratti gli embedding e aggregati secondo una delle seguenti strategie:
- Somma 
- Media 
- Prodotto

In [32]:
#funzione che restituisce gli embedding all'interno di un post
def get_post_embeddings(post, consider_pos=False):
    post_emb = []
    for token in post:
        word = token['word']
        pos = token['pos']
        if not consider_pos: 
            if word in embeddings:
                single_embedding = embeddings[word]
                post_emb.append(single_embedding)
        else:
            if word in embeddings and pos in ['ADJ', 'NOUN', 'VERB']:
                single_embedding = embeddings[word]
                post_emb.append(single_embedding)
    print(len(post_emb))
    if len(post_emb)==0:
        post_emb = [np.zeros(32)]
    return post_emb

def post_embeddings_sum(post_emb):
    return np.sum(post_emb, axis=0)

def post_embeddings_mean(post_emb):
    embeddings_sum = post_embeddings_sum(post_emb)
    return np.divide(embeddings_sum, len(post_emb))

def post_embeddings_prod(post_emb):
    return np.prod(post_emb, axis=0)

sample = get_post_embeddings(training_set[1], True)

print(f'Media: {post_embeddings_mean(sample)}')
print(f'Somma: {post_embeddings_sum(sample)}')
print(f'Prodotto: {post_embeddings_prod(sample)}')

5
Media: [-0.04242259 -0.03806086  0.04385353  0.03976631 -0.12742217  0.10998518
  0.0311     -0.06618695  0.06613407 -0.11457642  0.05476216 -0.08994417
 -0.0447743   0.0523218  -0.17183311  0.0963226  -0.01679959 -0.12544102
 -0.04104184 -0.09150524 -0.03717459  0.13664455 -0.19373289 -0.04618941
  0.06867969  0.0071115   0.02087701  0.03068707 -0.00392625 -0.07004991
  0.07070978  0.07404406]
Somma: [-0.21211296 -0.19030431  0.21926763  0.19883155 -0.63711086  0.54992588
  0.15549998 -0.33093473  0.33067035 -0.57288209  0.27381082 -0.44972083
 -0.22387151  0.261609   -0.85916553  0.481613   -0.08399795 -0.62720508
 -0.20520921 -0.4575262  -0.18587295  0.68322273 -0.96866444 -0.23094707
  0.34339846  0.0355575   0.10438506  0.15343533 -0.01963124 -0.35024957
  0.35354888  0.37022028]
Prodotto: [ 1.22333096e-05  1.83019025e-04  2.28736344e-05 -6.08206365e-07
  8.52695552e-05 -6.25187651e-05  1.77493883e-05  5.35576757e-06
 -3.33440727e-05 -6.89323128e-07  5.87005337e-04 -7.91464115e-